# Testing KAN (Kolmogorov-Arnold Networks) Model

This notebook demonstrates the use of Kolmogorov-Arnold Networks (KAN) for regression on wire drawing data.

KAN is a neural network architecture that leverages the Kolmogorov-Arnold representation theorem to approximate functions. This architecture can be particularly effective for regression tasks.

In this notebook, we will:
1. Load the wire drawing data
2. Create and configure a KAN model
3. Train and evaluate the model
4. Visualize the results
5. Analyze errors and save the model

In [13]:
import sys
import os
import pickle
import torch
sys.path.append('../../')

In [3]:
from analysis_functions import test_after_opt, split_transform_one_comp_cv, opener

In [4]:
# Load data
X_stress_components_new = opener(
    "X_stress_components_new_components", path_import="../../new_components_resources/"
)
y_stress_components_new = opener(
    "y_stress_components_new_components", path_import="../../new_components_resources/"
)

print(X_stress_components_new.shape)

component_num = 2
X_current = X_stress_components_new[component_num]
y_current = y_stress_components_new[component_num]

../../new_components_resources//X_stress_components_new_components.pkl
../../new_components_resources//y_stress_components_new_components.pkl
(3, 1597, 5)


In [5]:
from analysis_functions import KANModelTrainTest

In [6]:
kan_model = KANModelTrainTest(False)

2025-05-11 16:24:31,533 - INFO - Using device: cpu


In [7]:
kan_model.best_params = {
    "n_layers": 1,
    "opt": "LBFGS",
    "steps": 958,
    "grid": 1,
    "k": 2,
    "n_units_0": 57,
    "lr": 0.0403604924674922,
}

In [8]:
"""kan_model.create_train_val_test(
    X_current,
    y_current,
)
kan_model.calc_test_metric()"""

'kan_model.create_train_val_test(\n    X_current,\n    y_current,\n)\nkan_model.calc_test_metric()'

In [9]:
"""# 2. Save model state and parameters separately
torch.save(kan_model.final_model.state_dict(), f"{model_file_path}_state.pt")

# Save other necessary information
model_info = {
    "best_params": kan_model.best_params,
    "X_shape": X_current.shape,
    "y_shape": y_current.shape,
    "X_scaler": kan_model.scaler_X,
    "y_scaler": kan_model.scaler_y,
}

with open(f"{model_file_path}_info.pkl", "wb") as f:
    pickle.dump(model_info, f, protocol=4)

print(f"Model saved to {model_file_path}_state.pt and {model_file_path}_info.pkl")
"""

'# Create directory for saving model if it doesn\'t exist\nos.makedirs(\'saved_models\', exist_ok=True)\n\n# Save the entire KAN model object using pickle\nmodel_file_path = f\'saved_models/kan_model_component_{component_num}.pkl\'\n\nwith open(model_file_path, \'wb\') as f:\n    pickle.dump(kan_model, f)\n\nprint(f"KAN model saved to {model_file_path}")\n\n# To demonstrate how to load the model\nprint("\nExample of loading the model:")\nprint("with open(\'saved_models/kan_model_component_0.pkl\', \'rb\') as f:")\nprint("    loaded_model = pickle.load(f)")'

In [19]:
# Load model state and info
model_file_path = f'saved_models/kan_model_component_{component_num}'

# Check if files exist
if os.path.exists(f"{model_file_path}_state.pt") and os.path.exists(f"{model_file_path}.pkl_info.pkl"):
    # Load model info
    with open(f"{model_file_path}.pkl_info.pkl", "rb") as f:
        model_info = pickle.load(f)
    
    # Create a new KAN model with the same parameters
    loaded_kan_model = KANModelTrainTest(False)
    loaded_kan_model.best_params = model_info['best_params']
    
    # Create model architecture
    loaded_kan_model.create_train_val_test(
        X_current,
        y_current,
    )
    
    # Load the state dictionary
    loaded_kan_model.model.load_state_dict(torch.load(f"{model_file_path}_state.pt"))
    
    print("Model loaded successfully!")
    
    # Test the model if needed
    import torch
    import numpy as np
    
    # Get test data
    X_test = loaded_kan_model.cur_X_test
    y_test = loaded_kan_model.cur_y_test
    
    # Convert test data to tensor and move to device
    test_tensor = torch.tensor(X_test, dtype=torch.float32).to(loaded_kan_model.device)
    
    # Generate predictions with loaded model
    with torch.no_grad():
        loaded_predictions = loaded_kan_model.final_model(test_tensor).cpu().numpy().flatten()
    
    # Calculate RMSE
    rmse = np.sqrt(np.mean(np.square(loaded_predictions - y_test.flatten())))
    print(f"RMSE from loaded model: {rmse:.6f}")
else:
    print(f"Model files don't exist or are empty")

2025-05-11 16:39:37,440 - INFO - Using device: cpu


AttributeError: 'NoneType' object has no attribute 'load_state_dict'

In [11]:
# Load the pickled KAN model (demonstration - commented out to avoid duplication)
with open(model_file_path, 'rb') as f:
    loaded_kan_model = pickle.load(f)

# Test the loaded model (make predictions on test data)
import torch
import numpy as np

# Get test data
X_test = loaded_kan_model.cur_X_test
y_test = loaded_kan_model.cur_y_test

# Convert test data to tensor and move to device
test_tensor = torch.tensor(X_test, dtype=torch.float32).to(loaded_kan_model.device)

# Generate predictions with loaded model
with torch.no_grad():
    loaded_predictions = loaded_kan_model.final_model(test_tensor).cpu().numpy().flatten()

# Calculate RMSE
rmse = np.sqrt(np.mean(np.square(loaded_predictions - y_test.flatten())))
print(f"RMSE from loaded model: {rmse:.6f}")


# Note: In production, you can use the saved model as follows:
"""
from analysis_functions import opener

# Load the saved model
with open('saved_models/kan_model_component_0.pkl', 'rb') as f:
    model = pickle.load(f)

# Make predictions
def predict(input_features):
    # Convert input to tensor
    tensor_input = torch.tensor(input_features, dtype=torch.float32).to(model.device)
    
    # Generate predictions
    with torch.no_grad():
        predictions = model.final_model(tensor_input).cpu().numpy()
    
    return predictions
"""

EOFError: Ran out of input

## Model Saving Options

In this notebook, we've demonstrated saving the KAN model using pickle. There are several ways to save machine learning models:

1. **Pickle (used above)**: 
n   - Pros: Saves the entire object including all methods and attributes
   - Cons: Can be fragile across Python versions and requires the same dependencies

2. **PyTorch's save/load**:
   - Pros: More portable across PyTorch versions
   - Cons: Only saves model parameters, not the entire object structure

3. **ONNX format**:
   - Pros: Framework-agnostic, allows deployment across platforms
   - Cons: More complex to set up, may not preserve all KAN-specific features

Choose the method that best fits your deployment needs. For research and development within the same environment, pickle is often the simplest option.